In [125]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

sotu = pd.read_csv("http://cssbook.net/d/sotu.csv")
p_obama = sotu[sotu.President == "Obama"].text.str.split("\n\n").explode()
cv = CountVectorizer(min_df=.01, stop_words='english')
d = cv.fit_transform(p_obama)
d

<738x684 sparse matrix of type '<class 'numpy.int64'>'
	with 14396 stored elements in Compressed Sparse Row format>

In [158]:
from gensim import matutils
from gensim.models.ldamodel import LdaModel
corpus = matutils.Sparse2Corpus(d, documents_columns=False)
vocab = dict(enumerate(cv.get_feature_names()))

lda = LdaModel(corpus, num_topics=10, id2word=vocab, random_state=123, alpha='asymmetric')
pd.DataFrame({f"Topic {topicno}": [w for (w, tw) in words] 
              for (topicno, words) in lda.show_topics(formatted=False)})

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9
0,america,american,year,people,america,energy,new,america,america,new
1,american,jobs,americans,new,time,jobs,years,americans,tax,jobs
2,people,years,know,ve,ve,people,make,hard,work,just
3,new,like,people,just,new,new,jobs,people,states,help
4,right,let,new,american,year,years,work,years,americans,country
5,world,ve,american,make,years,america,need,nation,businesses,families
6,help,americans,years,time,security,just,people,work,god,make
7,economy,people,like,care,change,work,let,ve,bless,world
8,let,year,ve,like,working,world,america,world,united,government
9,know,home,make,country,work,clean,economy,time,people,fact


In [182]:
topics = pd.DataFrame([dict(lda.get_document_topics(doc, minimum_probability=0.0))
                      for doc in corpus])
topics.shape

(738, 10)

In [164]:
p_obama

74    Madam Speaker, Mr. Vice President, Members of ...
74    I have come here tonight not only to address t...
74    I know that for many Americans watching right ...
74    But while our economy may be weakened and our ...
74    The weight of this crisis will not determine t...
                            ...                        
81    I see it in the American who served his time, ...
81    I see it in the soldier who gives almost every...
81    I see it in the elderly woman who will wait in...
81    That's the America I know. That's the country ...
81    Thank you. God bless you. God bless the United...
Name: text, Length: 738, dtype: object

In [196]:
meta = sotu.iloc[p_obama.index].drop(columns=['text']).reset_index(drop=True)
tpd = pd.concat([meta, topics], axis=1)
tpd.sort_values(2, ascending=False).head()

,FirstName,President,Date,delivery,type,party,0,1,2,3,4,5,6,7,8,9
622,Barack,Obama,2015-01-20,spoken,SOTU,Democratic,0.004771,0.003538,0.979933,0.002375,0.002042,0.001792,0.001596,0.001439,0.001310,0.001203
11,Barack,Obama,2009-02-24,spoken,other,Democratic,0.005071,0.003796,0.978515,0.002548,0.002192,0.001923,0.001713,0.001545,0.001406,0.001291
322,Barack,Obama,2012-01-24,spoken,SOTU,Democratic,0.005116,0.003790,0.978478,0.002548,0.002192,0.001923,0.001713,0.001545,0.001406,0.001291
415,Barack,Obama,2013-02-12,spoken,SOTU,Democratic,0.005319,0.003985,0.977431,0.002679,0.002304,0.002022,0.001801,0.001624,0.001478,0.001357
370,Barack,Obama,2012-01-24,spoken,SOTU,Democratic,0.005468,0.004092,0.976827,0.002749,0.002365,0.002075,0.001848,0.001667,0.001517,0.001392


In [200]:
for docid in [622, 11, 322]:
    print(f"{docid}: {list(p_obama)[docid]}")

622: I intend to protect a free and open Internet, extend its reach to every classroom and every community and help folks build the fastest networks so that the next generation of digital innovators and entrepreneurs have the platform to keep reshaping our world. I want Americans to win the race for the kinds of discoveries that unleash new jobs: converting sunlight into liquid fuel; creating revolutionary prosthetics so that a veteran who gave his arms for his country can play catch with his kids again; pushing out into the solar system not just to visit, but to stay. Last month, we launched a new spacecraft as part of a reenergized space program that will send American astronauts to Mars. And in 2 months, to prepare us for those missions, Scott Kelly will begin a year-long stay in space. So good luck, Captain. Make sure to Instagram it. We're proud of you.
11: Because of this plan, there are teachers who can now keep their jobs and educate our kids, health care professionals can cont